In [ ]:
print("Compare Partition mechanism and Aziz")

Compare Partition mechanism and Aziz


In [ ]:
##Include##
import random
import numpy as np
import math
import statistics
import copy




debug=0

In [ ]:

def index_to_insert(denominator, i , sigma):
  random_num = random.random() 
  for j in range(1,i+1):
    Pij = pow(sigma, i-j)/denominator ## sigma^i-j/1+sigma+ ... + sigma ^ i-1
    ##print("P",i,j," = ",Pij)
    random_num = random_num - Pij     ## split 0-1 to intervals of 0-Pi1,Pi1-Pi2,....,Pii-1-Pii 
    if (random_num <=0) or (i==j):    ## check if random num is in interval Pij-1-Pij for defintion Pi0 = 0  
      return j-1                      ## 1-i but array index is from 0-i-1                     

def create_n_rannking_from_truthful_ranking_and_sigma(truthful_ranking, sigma=0.8):
  sampled_rank = []
  denominator = 1
  for (ranked_voter,i) in zip(truthful_ranking,range(1,len(truthful_ranking)+1)):
    index = index_to_insert(denominator, i, sigma)
    sampled_rank.insert(index, ranked_voter)   ## insert person truth rank number i to voter ranking in index
    denominator = denominator + pow(sigma,i)   ## Update denominator for next round denominator = 1+sigma+ ... + sigma ^ i-1
  return sampled_rank



print(create_n_rannking_from_truthful_ranking_and_sigma([3,1,2], sigma=1))

[2, 1, 3]


In [ ]:
def get_winners_percentage_from_voter_sampled_rank(voter_sampled_rank,num_of_winners):
  #num_of_winners = (len(voter_sampled_rank)*winners_percentage)//100
  return voter_sampled_rank[:num_of_winners]

In [ ]:
class partition:
  def __init__(self, num_of_partitions, num_of_voters, truthful_ranking=[],sigma=0,winners_percentage=3,num_of_reviews_per_candidate=7,split_step=0.5,first_step_winners=0.5,first_step_losers=0.7,prob_for_not_rank=0.3,new_method=False):
    self.winners_percentage = winners_percentage ## init winner percentage
    self.sigma = sigma  ## init sigma 
    self.num_of_partitions = num_of_partitions ## init num of_partitions
    self.num_of_voters = num_of_voters ## init num of voters
    self.voters_sampled_rank_by_partition = []  ## will use to keep for each partition each voter ranking
    self.num_of_reviews_per_candidate = num_of_reviews_per_candidate
    if not len(truthful_ranking):
      truthful_ranking = np.array(range(1, 201))
      np.random.shuffle(truthful_ranking)
    self.truthful_ranking = truthful_ranking
    self.voter_sampled_ranks = [create_n_rannking_from_truthful_ranking_and_sigma(truthful_ranking=np.delete(truthful_ranking, np.argwhere(truthful_ranking == voter)), sigma=sigma) for voter in range(1,num_of_voters+1)] ## Init ranks for each voter
    #self.voter_sampled_ranks = [create_n_rannking_from_truthful_ranking_and_sigma(truthful_ranking=np.delete(truthful_ranking, np.argwhere(truthful_ranking == voter)), sigma=sigma) for voter in range(1,num_of_voters+1)] ## Init ranks for each voter
    self.partition_number = num_of_voters//num_of_partitions
    modulo_of_winners = (num_of_voters*winners_percentage//100) % num_of_partitions
    self.num_of_winners = max(num_of_partitions,num_of_voters*winners_percentage//100 - modulo_of_winners)
    self.num_of_winners_per_partition = self.num_of_winners//num_of_partitions
    self.new_method = new_method
    self.split_step = split_step
    self.first_step_winners= first_step_winners
    self.first_step_losers = first_step_losers
    
    self.best_method = get_winners_percentage_from_voter_sampled_rank(truthful_ranking,self.num_of_winners)
    self.Voter_partition_i = [] ##voters partitions for debug
    self.iteration_i_unique = [] ## for more information
    self.iteration_i_counts = [] ## for more information
    self.itreration_train_vote_x_times = [] ## for more information
    self.partition_votes_to_candidates = []

    self.num_of_first_votes = max(1,math.ceil(self.num_of_reviews_per_candidate*self.split_step))
    self.num_of_first_votes_winners = math.ceil(self.num_of_winners*self.first_step_winners)
    self.num_of_first_votes_losers = math.ceil(self.num_of_winners*self.first_step_losers)
    self.num_of_first_votes_winners_per_partition = self.num_of_first_votes_winners//self.num_of_partitions
    self.num_of_first_votes_losers_per_partition = self.num_of_first_votes_losers//self.num_of_partitions
    self.num_of_second_votes = self.num_of_reviews_per_candidate - self.num_of_first_votes
    self.prob_for_not_rank = prob_for_not_rank



  def get_partition_winners(self,partition_votes,keep_log=False,draw_method="random"):
    unique, counts = np.unique(partition_votes, return_counts=True)  
    num_of_winners = len(partition_votes[0]) ##each voter give num of winners votes so num_of_winners == num of each voter

    if debug:
      print("Uniqe : " , unique)
      print("Counts : " ,counts)
      print("Sum of Counts : ", sum(counts))
      print("Choose : " , [x for _, x in sorted(zip(counts, unique), reverse=True)][:num_of_winners])
    
    if keep_log:
      self.iteration_i_unique.append(unique)
      self.iteration_i_counts.append(counts)

    if draw_method == "random":
      winners = sorted(zip(counts, unique), reverse=True)
      split_data_to_groups_by_count = []
      currentGroup = []
      last_count = winners[0][0]
      for countVote in winners:
        if last_count == countVote[0]:
          currentGroup.append(countVote[1])
        else:
          last_count = countVote[0]
          split_data_to_groups_by_count.append(currentGroup)
          currentGroup = [countVote[1]]
      if currentGroup != []:
        split_data_to_groups_by_count.append(currentGroup)

      if debug:
        print("Split_data : " , split_data_to_groups_by_count)

      num_of_winners_until_now = 0
      ret = []
      for gruopCount in split_data_to_groups_by_count:
        if num_of_winners_until_now == num_of_winners:
          return ret
        num_of_over_winners_in_group = len(gruopCount) + num_of_winners_until_now - num_of_winners
        if num_of_over_winners_in_group > 0:
          num_of_winners_to_add = num_of_winners - num_of_winners_until_now
          while (num_of_winners_to_add > 0):
            random_num = random.choice(gruopCount)
            if debug:
              print("Random Number : ", random_num)
            gruopCount.remove(random_num)
            ret.append(random_num)
            num_of_winners_to_add -= 1
          return ret
        else:
          num_of_winners_until_now += len(gruopCount)
          ret.extend(gruopCount)

    return [x for _, x in sorted(zip(counts, unique), reverse=True)][:num_of_winners]

  def train(self):
    self.voters_sampled_rank_by_partition = []
    self.Voter_partition_i = []
    Voter_partition = np.array(range(1,self.num_of_voters+1))
    np.random.shuffle(Voter_partition)
    if debug:
      print("num_of_voters : ", self.num_of_voters, " num_of_partitions :", self.num_of_partitions)
    num_of_voters_in_group = self.num_of_voters//self.num_of_partitions
    for partition_index in range(self.num_of_partitions):
      Voter_partition_i = Voter_partition[partition_index*num_of_voters_in_group:(partition_index+1)*num_of_voters_in_group]
      Voter_partition_i = sorted(Voter_partition_i, key=self.truthful_ranking.tolist().index) ##.sort sort by thruthful 
      self.Voter_partition_i.append(Voter_partition_i)
      if debug:
        print("Voter_partition[", partition_index , "] : ", Voter_partition_i)
      self.voters_sampled_rank_by_partition.append([[b for b in self.voter_sampled_ranks[x-1] if b not in Voter_partition_i] for x in Voter_partition_i]) ## At x-1 we have the rank for voter x

  def reSampled(self):
    self.voter_sampled_ranks = [create_n_rannking_from_truthful_ranking_and_sigma(truthful_ranking=np.delete(truthful_ranking, np.argwhere(truthful_ranking == voter)), sigma=sigma) for voter in range(1,num_of_voters+1)] ## Init ranks for each voter

  def get_rotate_array(self,original_list,num_of_rotate):
    copied_list = original_list[:]
    new_array = original_list[num_of_rotate:] + original_list[:num_of_rotate]
    return new_array

  def vote_same_votes(self):
    partition_winners = []
    partitions_score = {}
    votes_map = {}
    for i in range(self.num_of_reviews_per_candidate):
      for partition_index in range(self.num_of_partitions):
        #print(self.voters_sampled_rank_by_partition[partition_index])
        # print(self.Voter_partition_i[partition_index])
        if i == 0:
          partitions_score[partition_index] = {}
        voter_partition = self.get_rotate_array(self.Voter_partition_i[self.num_of_partitions-partition_index-1], i)
        for candidate, voter_rank, voter_name in zip(voter_partition,self.voters_sampled_rank_by_partition[partition_index],self.Voter_partition_i[partition_index]):
          vote = (1-(voter_rank.index(candidate)/len(voter_rank)))*100
          if i == 0:
            partitions_score[partition_index][candidate] = {}
            partitions_score[partition_index][candidate]['votes'] = [vote]
            partitions_score[partition_index][candidate]['mean'] = statistics.mean(partitions_score[partition_index][candidate]['votes']) #vote/self.num_of_reviews_per_candidate
            partitions_score[partition_index][candidate]['rank_by'] = [voter_name]
          else:
            partitions_score[partition_index][candidate]['votes'].append(vote)
            partitions_score[partition_index][candidate]['mean'] = statistics.mean(partitions_score[partition_index][candidate]['votes']) #vote/self.num_of_reviews_per_candidate
            partitions_score[partition_index][candidate]['rank_by'].append(voter_name)
    # print(self.voters_sampled_rank_by_partition[0])
    # print(partitions_score[0])
    # print(partitions_score[1])
    partition_0_winners = list(map(lambda x: x[0],sorted(partitions_score[0].items(), key=lambda item: 100- item[1]["mean"])[0:self.num_of_winners_per_partition]))
    partition_1_winners = list(map(lambda x: x[0],sorted(partitions_score[1].items(), key=lambda item: 100- item[1]["mean"])[0:self.num_of_winners_per_partition]))
    partition_winners.extend(partition_0_winners)
    partition_winners.extend(partition_1_winners)
    return partition_winners

  def get_not_vote_list(self,partitions_score, num_of_winners, num_of_losers, prob):
    not_rank_second_round = []
    partition_winners_first_round = []
    partition_losers_first_round = []


    sorted_partition_0 = list(map(lambda x: x[0],sorted(partitions_score[0].items(), key=lambda item: 100- item[1]["mean"])))
    sorted_partition_1 = list(map(lambda x: x[0],sorted(partitions_score[1].items(), key=lambda item: 100- item[1]["mean"])))
    
    partition_0_winners = sorted_partition_0[0:num_of_winners]
    partition_1_winners = sorted_partition_1[0:num_of_winners]
    need_to_pop = random.random()
    if need_to_pop < prob and num_of_winners > 0:
      partition_0_winners.pop()
      partition_1_winners.pop()


    partition_0_losers = sorted_partition_0[-1*num_of_losers:]
    partition_1_losers = sorted_partition_1[-1*num_of_losers:]
    partition_winners_first_round.extend(partition_0_winners)
    partition_winners_first_round.extend(partition_1_winners)

    partition_losers_first_round.extend(partition_0_losers)
    partition_losers_first_round.extend(partition_1_losers)

    not_rank_second_round.extend(partition_winners_first_round)
    not_rank_second_round.extend(partition_losers_first_round)

    return not_rank_second_round


  def vote_not_same_votes(self):
    partition_winners_first_round = []
    partition_losers_first_round = []
    partition_winners_second_round = []
    partition_winners = []
    not_rank_second_round = []
    partitions_score = {}
    num_of_first_votes = self.num_of_first_votes #math.floor(self.num_of_reviews_per_candidate*self.split_step)
    num_of_first_votes_winners = self.num_of_first_votes_winners #math.floor(self.num_of_winners*self.first_step_winners)
    num_of_first_votes_losers = self.num_of_first_votes_losers #math.floor(self.num_of_winners*self.first_step_losers)
    num_of_first_votes_winners_per_partition = self.num_of_first_votes_winners_per_partition #num_of_first_votes_winners//self.num_of_partitions
    num_of_first_votes_losers_per_partition = self.num_of_first_votes_losers_per_partition #num_of_first_votes_losers//self.num_of_partitions
    num_of_second_votes = self.num_of_second_votes #self.num_of_reviews_per_candidate - num_of_first_votes

    #print("Num of first votes winners: ",num_of_first_votes_losers_per_partition)
    #print("Num of first step rounds: ", num_of_first_votes)

    for i in range(num_of_first_votes):
      for partition_index in range(self.num_of_partitions):
        if i == 0:
          partitions_score[partition_index] = {}
        # print(self.Voter_partition_i[self.num_of_partitions-partition_index-1])
        voter_partition = self.get_rotate_array(self.Voter_partition_i[self.num_of_partitions-partition_index-1], i)
        for candidate, voter_rank, voter_name in zip(voter_partition,self.voters_sampled_rank_by_partition[partition_index],self.Voter_partition_i[partition_index]):
          vote = (1-(voter_rank.index(candidate)/len(voter_rank)))*100
          if i == 0:
            partitions_score[partition_index][candidate] = {}
            partitions_score[partition_index][candidate]['votes'] = [vote]
            partitions_score[partition_index][candidate]['mean'] = statistics.mean(partitions_score[partition_index][candidate]['votes']) #vote/num_of_first_votes
            partitions_score[partition_index][candidate]['rank_by'] = [voter_name]
            partitions_score[partition_index][candidate]['num_of_votes'] = 1
          else:
            partitions_score[partition_index][candidate]['votes'].append(vote)
            partitions_score[partition_index][candidate]['mean'] = statistics.mean(partitions_score[partition_index][candidate]['votes']) #vote/num_of_first_votes
            partitions_score[partition_index][candidate]['rank_by'].append(voter_name)
            partitions_score[partition_index][candidate]['num_of_votes'] += 1
    # print(partitions_score[0])
    # print(partitions_score[1])

    #print(list(map(lambda x: x[0],sorted(partitions_score[0].items(), key=lambda item: 100- item[1]["mean"]))))
    #print(list(map(lambda x: x[0],sorted(partitions_score[1].items(), key=lambda item: 100- item[1]["mean"]))))
    sorted_partition_0 = list(map(lambda x: x[0],sorted(partitions_score[0].items(), key=lambda item: 100- item[1]["mean"])))
    sorted_partition_1 = list(map(lambda x: x[0],sorted(partitions_score[1].items(), key=lambda item: 100- item[1]["mean"])))
    # print(sorted_partition_0)
    partition_0_winners = sorted_partition_0[0:num_of_first_votes_winners_per_partition]
    partition_1_winners = sorted_partition_1[0:num_of_first_votes_winners_per_partition]
    partition_0_losers = sorted_partition_0[-1*num_of_first_votes_losers_per_partition:]
    partition_1_losers = sorted_partition_1[-1*num_of_first_votes_losers_per_partition:]
    partition_winners_first_round.extend(partition_0_winners)
    partition_winners_first_round.extend(partition_1_winners)

    partition_losers_first_round.extend(partition_0_losers)
    partition_losers_first_round.extend(partition_1_losers)

    not_rank_second_round.extend(partition_winners_first_round)
    not_rank_second_round.extend(partition_losers_first_round)

    prob_for_not_rank = self.prob_for_not_rank
    
    # print("Winners first round: ", partition_winners_first_round)
    # print("Losers first round: ", partition_losers_first_round)
    # print("Not rank: ", not_rank_second_round)

    # print("From function not rank: ", self.get_not_vote_list(partitions_score,num_of_first_votes_winners_per_partition,num_of_first_votes_losers_per_partition))

    for i in range(num_of_first_votes,self.num_of_reviews_per_candidate+1):
      for partition_index in range(self.num_of_partitions):
        voter_partition = self.get_rotate_array(self.Voter_partition_i[self.num_of_partitions-partition_index-1], i)
        choosed_voters = []
        for candidate, voter_rank, voter_name in zip(voter_partition,self.voters_sampled_rank_by_partition[partition_index],self.Voter_partition_i[partition_index]):
          num_of_retries = 0
          if (candidate in not_rank_second_round) or (voter_name in partitions_score[partition_index][candidate]['rank_by']):
            candidate = random.choice(voter_partition)
            while (voter_name in partitions_score[partition_index][candidate]['rank_by']) or (candidate in choosed_voters) or (candidate in not_rank_second_round):
              num_of_retries+=1
              candidate = random.choice(voter_partition)
              if (num_of_retries == 10):
                break
              # print("Candidate: ", candidate)
              # print("Voter_name: ", voter_name, " Rank_by: ", partitions_score[partition_index][candidate]['rank_by'])
              # print("choosed_voters: ", choosed_voters)
              # print("not_rank_second_round: ", not_rank_second_round)
            if (num_of_retries == 10):
              continue
            choosed_voters.append(candidate)
          if (num_of_retries == 10):
            print("Bugg ", voter_name)
          vote = (1-(voter_rank.index(candidate)/len(voter_rank)))*100
          partitions_score[partition_index][candidate]['votes'].append(vote)
          partitions_score[partition_index][candidate]['mean'] = statistics.mean(partitions_score[partition_index][candidate]['votes']) #vote/num_of_first_votes
          partitions_score[partition_index][candidate]['rank_by'].append(voter_name)
          partitions_score[partition_index][candidate]['num_of_votes'] += 1
      # print("not_rank_second_round ", i , " : ", not_rank_second_round)
      # print("\n", self.voters_sampled_rank_by_partition[0])
      # print(partitions_score[0],"\n")
      # print("\n",self.voters_sampled_rank_by_partition[1])
      # print(partitions_score[1],"\n")
      not_rank_second_round = self.get_not_vote_list(partitions_score,num_of_first_votes_winners_per_partition,num_of_first_votes_losers_per_partition,prob_for_not_rank)

    #print(partitions_score[0])
    #print(partitions_score[1])
    sorted_partition_0 = list(map(lambda x: x[0],sorted(partitions_score[0].items(), key=lambda item: 100- item[1]["mean"])))
    sorted_partition_1 = list(map(lambda x: x[0],sorted(partitions_score[1].items(), key=lambda item: 100- item[1]["mean"])))

    partition_0_winners = sorted_partition_0[0:self.num_of_winners_per_partition]
    partition_1_winners = sorted_partition_1[0:self.num_of_winners_per_partition]
    partition_winners.extend(partition_0_winners)
    partition_winners.extend(partition_1_winners)


    return partition_winners

  # def vote(self):
  #   if self.new_method:
  #     return self.vote_not_same_votes()
  #   else:
  #     return self.vote_same_votes()

  def best_vote(self):
    winners = []
    for partition_index in range(self.num_of_partitions):
      winners.extend(self.Voter_partition_i[partition_index][0:4])
    
    return winners

  def votes_all_candidates(self):
    partition_winners = []
    partitions_score = {}
    for partition_index in range(self.num_of_partitions):
      partitions_score[partition_index] = {}
      i=0
      for voter_rank, voter_name in zip(self.voters_sampled_rank_by_partition[partition_index],self.Voter_partition_i[partition_index]):
          for candidate in voter_rank:
            if i==0:
              partitions_score[partition_index][candidate] = {}
              partitions_score[partition_index][candidate]['votes'] = []
              partitions_score[partition_index][candidate]['mean'] = 0
              partitions_score[partition_index][candidate]['rank_by'] = []
              partitions_score[partition_index][candidate]['num_of_votes'] = 0

            vote = (1-(voter_rank.index(candidate)/len(voter_rank)))*100
            partitions_score[partition_index][candidate]['votes'].append(vote)
            partitions_score[partition_index][candidate]['mean'] = statistics.mean(partitions_score[partition_index][candidate]['votes']) #vote/num_of_first_votes
            partitions_score[partition_index][candidate]['rank_by'].append(voter_name)
            partitions_score[partition_index][candidate]['num_of_votes'] += 1
          i+=1
    #print("partition i: " ,self.voters_sampled_rank_by_partition[partition_index])
    sorted_partition_0 = list(map(lambda x: x[0],sorted(partitions_score[0].items(), key=lambda item: 100- item[1]["mean"])))
    sorted_partition_1 = list(map(lambda x: x[0],sorted(partitions_score[1].items(), key=lambda item: 100- item[1]["mean"])))


    #print("num of winners per partition: ", self.num_of_winners_per_partition)
    partition_0_winners = sorted_partition_0[0:self.num_of_winners_per_partition]
    partition_1_winners = sorted_partition_1[0:self.num_of_winners_per_partition]
    partition_winners.extend(partition_0_winners)
    partition_winners.extend(partition_1_winners)

    #print("All vote partition 0: ", self.voters_sampled_rank_by_partition[0])
    #print("All vote partition 1: ", self.voters_sampled_rank_by_partition[1])

    #print("All vote: ", partitions_score[0])
    #print("All vote: ", partitions_score[1])
    return partition_winners

  def vote(self, get_best_performe=False):
    return (self.best_vote(), self.votes_all_candidates(), self.vote_not_same_votes(), self.vote_same_votes())
    if self.new_method:
      return self.vote_not_same_votes()
    else:
      return self.vote_same_votes()

  def train_vote(self, get_best_performe=False):
    self.train()
    return self.vote(get_best_performe)

  def train_vote_x_times(self, num_of_iterations):
    iterations_votes = [self.train_vote() for _ in range(num_of_iterations)]  ##runs num_of_iterations different partition and store their votes
    return self.get_partition_winners(iterations_votes)

  def train_vote_x_times_score(self, num_of_iterations,best_method=[],metrics=["avg","dis", "avgDis"],resmapled=True):
    if resmapled:
      self.reSampled()  ##resampled voters ranks 
    if not len(best_method):
      best_method = self.best_method
    ret = {}
    for metric in metrics: ##Init ret
      ret[metric] = 0

    iterations_votes = [self.train_vote() for _ in range(num_of_iterations)]  ##runs num_of_iterations different partition and store their vote
    self.itreration_train_vote_x_times.append(self.get_partition_winners(iterations_votes,keep_log=True))

    for i in range(num_of_iterations):
      acc_dic = self.accuray(iterations_votes[i],best_method=best_method,metrics=metrics)
      for metric in metrics:
        ret[metric] += acc_dic[metric]

    for metric in metrics:
        ret[metric] = ret[metric]/num_of_iterations

    return ret

  def train_vote_x_times_score_resmapled(self, num_of_iterations,resmapled_every=5,best_method=[],metrics=["avg","dis", "avgDis"]):
    if not len(best_method):
      best_method = self.best_method

    ret = {}
    for metric in metrics: ##Init ret
      ret[metric] = 0

    for _ in range(resmapled_every):
      x_time_score = self.train_vote_x_times_score(num_of_iterations=num_of_iterations,best_method=best_method,metrics=metrics,resmapled=True)
      print("x_time_score: ", x_time_score)
      for metric in metrics:
        ret[metric] += x_time_score[metric]

    for metric in metrics:
        ret[metric] = ret[metric]/resmapled_every

    return ret
    
    #iterations_votes_scores = [self.train_vote_x_times_score(num_of_iterations=num_of_iterations,best_method=best_method,metrics=metrics,resmapled=True) for _ in range(resmapled_every)]  ##runs num_of_iterations different partition and store their votes
    #return sum(iterations_votes_scores)/num_of_iterations

  def calc_distance_between_votes(self, first_vote,second_vote):
    return self.truthful_ranking.tolist().index(first_vote) - self.truthful_ranking.tolist().index(second_vote)

  def accuracy(self, votes, best_method=[],metrics=["avg","dis", "avgDis"]):
    if not len(best_method):
      best_method = self.best_method
    ret = {}
    wrong_votes = [x for x in votes if x not in best_method]
    missing_best = [x for x in best_method if x not in votes]
    num_of_winners = len(best_method)
    #print("self.num_of_winners: ", self.num_of_winners)
    missing_votes_avg = len(wrong_votes)/num_of_winners
    if "avg" in metrics:
      ret["avg"] = missing_votes_avg

    if "dis" in metrics or "avgDis" in metrics:
      distance_on_error = 0
      for wrong_vote, miss_vote in zip(wrong_votes, missing_best):
        distance_between_vote = self.calc_distance_between_votes(wrong_vote, miss_vote)
        distance_on_error+= distance_between_vote
      if "dis" in metrics:
        ret["dis"] = distance_on_error
      if "avgDis" in metrics:
        ret["avgDis"] = missing_votes_avg*distance_on_error
    
    return ret

  def print_test_summary(self):
    print("\nSummary ")
    print("\nphi: ", self.sigma)
    print("\nnum of voters: ", self.num_of_voters)
    print("\nnum of winners: ", self.num_of_winners)
    print("\nnum_of_reviews_per_candidate: ", self.num_of_reviews_per_candidate)
    print("\nnum of first votes: ", self.num_of_first_votes)
    print("\nnum_of_second_votes: ", self.num_of_second_votes)
    print("\nnum_of_first_votes_winners: ",self.num_of_first_votes_winners)
    print("\nnum_of_first_votes_losers: ", self.num_of_first_votes_losers)
    print("\nprob_for_not_rank: ", self.prob_for_not_rank)
    print("\n\n\n")


In [ ]:
phi = [0.75, 0.8, 0.85, 0.9]
num_of_voters = [20, 30, 40, 50, 60, 70, 80]
winners_percentage = [5, 10, 15, 20, 25, 30, 35]
num_of_reviews_per_candidate=[5, 7, 9, 11]
split_step=[0, 0.25, 0.5, 0.75]
first_step_winners=[0, 0.25, 0.5, 0.75]
first_step_losers=[0, 0.25, 0.5, 0.75]
prob_for_not_rank = [0.1, 0.3, 0.5, 0.7]


num_of_iterations = 1000
num_of_tests = 50

color = ""
success_params = []
failed_params = []

num_of_new_method_win = 0
num_of_old_method_win = 0
num_of_no_winner = 0

for test_num in range(1, num_of_tests+1):
  
  best_avg_dis = 0
  best_avg_avg = 0
  best_avg_avg_Dis = 0

  all_vote_avg_dis = 0
  all_vote_avg_avg = 0
  all_vote_avg_avg_Dis = 0

  new_avg_dis = 0
  new_avg_avg = 0
  new_avg_avg_Dis = 0

  old_avg_dis = 0
  old_avg_avg = 0
  old_avg_avg_Dis = 0

  phi_val = random.choice(phi)
  num_of_voters_val = random.choice(num_of_voters)
  winners_percentage_val = random.choice(winners_percentage)
  num_of_reviews_per_candidate_val = random.choice(num_of_reviews_per_candidate)
  split_step_val = random.choice(split_step)
  first_step_winners_val = random.choice(first_step_winners)
  first_step_losers_val = random.choice(first_step_losers)
  prob_val = random.choice(prob_for_not_rank)


  for i in range(num_of_iterations):
    #print("\nIteartion number ", i, "\n")
    partition_class_test = partition(2,num_of_voters=num_of_voters_val,truthful_ranking=np.array(range(1,num_of_voters_val+1)),sigma=phi_val,winners_percentage=winners_percentage_val,num_of_reviews_per_candidate=num_of_reviews_per_candidate_val,split_step=split_step_val,first_step_winners=first_step_winners_val,first_step_losers=first_step_losers_val,prob_for_not_rank=prob_val,new_method=True)
    best_method_winners, all_votes, new_method_winners, old_method_winners = partition_class_test.train_vote()
    #print("new_method_winners: ", new_method_winners, " best_method_winners: ", best_method_winners, "all_votes: ", all_votes)

    best_avg, best_dis, best_avg_Dis = partition_class_test.accuracy(best_method_winners).values()
    best_avg_dis+= best_dis
    best_avg_avg+= best_avg
    best_avg_avg_Dis += best_avg_Dis

    all_vote_avg, all_vote_dis, all_vote_avg_Dis = partition_class_test.accuracy(all_votes).values()
    all_vote_avg_dis+= all_vote_dis
    all_vote_avg_avg+= all_vote_avg
    all_vote_avg_avg_Dis += all_vote_avg_Dis

    new_avg, new_dis, new_avg_Dis = partition_class_test.accuracy(new_method_winners).values()
    new_avg_dis+= new_dis
    new_avg_avg+= new_avg
    new_avg_avg_Dis += new_avg_Dis
    #print("dis ", new_dis, " avg ", new_avg, " avgDis ", new_avg_Dis)

    old_avg, old_dis, old_avg_Dis = partition_class_test.accuracy(old_method_winners).values()
    old_avg_dis+= old_dis
    old_avg_avg+= old_avg
    old_avg_avg_Dis += old_avg_Dis
  if (new_avg_dis < old_avg_dis):
    success_params.append({'sigma:': phi_val, 'num_of_voters: ' : num_of_voters_val, 'winners_percentage: ' : winners_percentage_val, 'num_of_reviews_per_candidate: ' : num_of_reviews_per_candidate_val, 'split_step: ' : split_step_val ,'first_step_winners: ' : first_step_winners_val, 'first_step_losers: ' : first_step_losers_val, 'prob_val: ' : prob_val })
    num_of_new_method_win += 1
    color = '\033[92m'
  elif (new_avg_dis == old_avg_dis):
    num_of_no_winner+=1
    color = '\033[93m'
  else:
    num_of_old_method_win += 1
    color = '\033[91m'
    failed_params.append({'sigma:': phi_val, 'num_of_voters: ' : num_of_voters_val, 'winners_percentage: ' : winners_percentage_val, 'num_of_reviews_per_candidate: ' : num_of_reviews_per_candidate_val, 'split_step: ' : split_step_val ,'first_step_winners: ' : first_step_winners_val, 'first_step_losers: ' : first_step_losers_val})

  print("\n")
  print(color+"Test number: ", test_num)
  #print("\nSummary for phi: ", phi_val, " and Num of iterations: ", num_of_iterations, " num of winners: ", partition_class_test.num_of_winners, " num of first votes: ", partition_class_test.num_of_first_votes)
  partition_class_test.print_test_summary()
  print("Best method:")
  print("Avg dis: ", best_avg_dis/num_of_iterations)
  print("Avg avg: ", best_avg_avg/num_of_iterations)
  print("Avg avgDis: ", best_avg_avg_Dis/num_of_iterations)

  print("\nall Vote method:")
  print("Avg dis: ", all_vote_avg_dis/num_of_iterations)
  print("Avg avg: ", all_vote_avg_avg/num_of_iterations)
  print("Avg avgDis: ", all_vote_avg_avg_Dis/num_of_iterations)

  print("\nNew method:")
  print("Avg dis: ", new_avg_dis/num_of_iterations)
  print("Avg avg: ", new_avg_avg/num_of_iterations)
  print("Avg avgDis: ", new_avg_avg_Dis/num_of_iterations)

  print("\nOld method:")
  print("Avg dis: ", old_avg_dis/num_of_iterations)
  print("Avg avg: ", old_avg_avg/num_of_iterations)
  print("Avg avgDis: ", old_avg_avg_Dis/num_of_iterations)

  

print('\033[0m' + "Number of time new method was better: ", num_of_new_method_win, " number of time old method was better: ", num_of_old_method_win, " number of time of same score: ", num_of_no_winner)
print(success_params)
print("/n/n/n/n")
print(failed_params)



Test number:  1

Summary 

phi:  0.85

num of voters:  50

num of winners:  6

num_of_reviews_per_candidate:  7

num of first votes:  2

num_of_second_votes:  5

num_of_first_votes_winners:  5

num_of_first_votes_losers:  3

prob_for_not_rank:  0.3




Best method:
Avg dis:  0.498
Avg avg:  0.3713333333333307
Avg avgDis:  0.27466666666666667

all Vote method:
Avg dis:  5.019
Avg avg:  0.21116666666666636
Avg avgDis:  1.4943333333333324

New method:
Avg dis:  10.448
Avg avg:  0.3118333333333341
Avg avgDis:  3.966666666666666

Old method:
Avg dis:  9.761
Avg avg:  0.30516666666666653
Avg avgDis:  3.6365000000000007


Test number:  2

Summary 

phi:  0.85

num of voters:  80

num of winners:  12

num_of_reviews_per_candidate:  9

num of first votes:  5

num_of_second_votes:  4

num_of_first_votes_winners:  3

num_of_first_votes_losers:  6

prob_for_not_rank:  0.1




Best method:
Avg dis:  1.054
Avg avg:  0.011500000000000005
Avg avgDis:  0.13174999999999998

all Vote method:
Avg dis:  

In [ ]:
phi = [0.75, 0.8, 0.85, 0.9]
num_of_voters = [20, 30, 40, 50, 60, 70, 80]
winners_percentage = [5, 10, 15, 20, 25, 30, 35]
num_of_reviews_per_candidate=[5, 7, 9, 11]
split_step=[0.25, 0.5, 0.75]
first_step_winners=[0.25, 0.5, 0.75]
first_step_losers=[0.25, 0.5, 0.75]
prob_for_not_rank = [0.1, 0.3, 0.5, 0.7]



num_of_iterations = 1000
num_of_tests = 50

color = ""

success_params=[]
failed_params = []

num_of_new_method_win = 0
num_of_old_method_win = 0
num_of_no_winner = 0

for test_num in range(1, num_of_tests+1):
  
  best_avg_dis = 0
  best_avg_avg = 0
  best_avg_avg_Dis = 0

  all_vote_avg_dis = 0
  all_vote_avg_avg = 0
  all_vote_avg_avg_Dis = 0

  new_avg_dis = 0
  new_avg_avg = 0
  new_avg_avg_Dis = 0

  old_avg_dis = 0
  old_avg_avg = 0
  old_avg_avg_Dis = 0

  phi_val = random.choice(phi)
  num_of_voters_val = random.choice(num_of_voters)
  winners_percentage_val = random.choice(winners_percentage)
  num_of_reviews_per_candidate_val = random.choice(num_of_reviews_per_candidate)
  split_step_val = random.choice(split_step)
  first_step_winners_val = random.choice(first_step_winners)
  first_step_losers_val = random.choice(first_step_losers)
  prob_val = random.choice(prob_for_not_rank)


  for i in range(num_of_iterations):
    #print("\nIteartion number ", i, "\n")
    partition_class_test = partition(2,num_of_voters=num_of_voters_val,truthful_ranking=np.array(range(1,num_of_voters_val+1)),sigma=phi_val,winners_percentage=winners_percentage_val,num_of_reviews_per_candidate=num_of_reviews_per_candidate_val,split_step=split_step_val,first_step_winners=first_step_winners_val,first_step_losers=first_step_losers_val,prob_for_not_rank=prob_val,new_method=True)
    best_method_winners, all_votes, new_method_winners, old_method_winners = partition_class_test.train_vote()
    #print("new_method_winners: ", new_method_winners, " best_method_winners: ", best_method_winners, "all_votes: ", all_votes)

    best_avg, best_dis, best_avg_Dis = partition_class_test.accuracy(best_method_winners).values()
    best_avg_dis+= best_dis
    best_avg_avg+= best_avg
    best_avg_avg_Dis += best_avg_Dis

    all_vote_avg, all_vote_dis, all_vote_avg_Dis = partition_class_test.accuracy(all_votes).values()
    all_vote_avg_dis+= all_vote_dis
    all_vote_avg_avg+= all_vote_avg
    all_vote_avg_avg_Dis += all_vote_avg_Dis

    new_avg, new_dis, new_avg_Dis = partition_class_test.accuracy(new_method_winners).values()
    new_avg_dis+= new_dis
    new_avg_avg+= new_avg
    new_avg_avg_Dis += new_avg_Dis
    #print("dis ", new_dis, " avg ", new_avg, " avgDis ", new_avg_Dis)

    old_avg, old_dis, old_avg_Dis = partition_class_test.accuracy(old_method_winners).values()
    old_avg_dis+= old_dis
    old_avg_avg+= old_avg
    old_avg_avg_Dis += old_avg_Dis
  if (new_avg_dis < old_avg_dis):
    success_params.append({'sigma:': phi_val, 'num_of_voters: ' : num_of_voters_val, 'winners_percentage: ' : winners_percentage_val, 'num_of_reviews_per_candidate: ' : num_of_reviews_per_candidate_val, 'split_step: ' : split_step_val ,'first_step_winners: ' : first_step_winners_val, 'first_step_losers: ' : first_step_losers_val, 'prob_val: ' : prob_val })
    num_of_new_method_win += 1
    color = '\033[92m'
  elif (new_avg_dis == old_avg_dis):
    num_of_no_winner+=1
    color = '\033[93m'
  else:
    num_of_old_method_win += 1
    failed_params.append({'sigma:': phi_val, 'num_of_voters: ' : num_of_voters_val, 'winners_percentage: ' : winners_percentage_val, 'num_of_reviews_per_candidate: ' : num_of_reviews_per_candidate_val, 'split_step: ' : split_step_val ,'first_step_winners: ' : first_step_winners_val, 'first_step_losers: ' : first_step_losers_val})
    color = '\033[91m'
  print("\n")
  print(color+"Test number: ", test_num)
  #print("\nSummary for phi: ", phi_val, " and Num of iterations: ", num_of_iterations, " num of winners: ", partition_class_test.num_of_winners, " num of first votes: ", partition_class_test.num_of_first_votes)
  partition_class_test.print_test_summary()
  print("Best method:")
  print("Avg dis: ", best_avg_dis/num_of_iterations)
  print("Avg avg: ", best_avg_avg/num_of_iterations)
  print("Avg avgDis: ", best_avg_avg_Dis/num_of_iterations)

  print("\nall Vote method:")
  print("Avg dis: ", all_vote_avg_dis/num_of_iterations)
  print("Avg avg: ", all_vote_avg_avg/num_of_iterations)
  print("Avg avgDis: ", all_vote_avg_avg_Dis/num_of_iterations)

  print("\nNew method:")
  print("Avg dis: ", new_avg_dis/num_of_iterations)
  print("Avg avg: ", new_avg_avg/num_of_iterations)
  print("Avg avgDis: ", new_avg_avg_Dis/num_of_iterations)

  print("\nOld method:")
  print("Avg dis: ", old_avg_dis/num_of_iterations)
  print("Avg avg: ", old_avg_avg/num_of_iterations)
  print("Avg avgDis: ", old_avg_avg_Dis/num_of_iterations)

  

print('\033[0m' + "Number of time new method was better: ", num_of_new_method_win, " number of time old method was better: ", num_of_old_method_win, " number of time of same score: ", num_of_no_winner)
print(success_params)
print("/n/n/n/n")
print(failed_params)



Test number:  1

Summary 

phi:  0.85

num of voters:  50

num of winners:  12

num_of_reviews_per_candidate:  11

num of first votes:  6

num_of_second_votes:  5

num_of_first_votes_winners:  9

num_of_first_votes_losers:  9

prob_for_not_rank:  0.7




Best method:
Avg dis:  0.753
Avg avg:  0.008666666666666663
Avg avgDis:  0.08283333333333333

all Vote method:
Avg dis:  6.722
Avg avg:  0.12299999999999973
Avg avgDis:  1.1861666666666673

New method:
Avg dis:  8.495
Avg avg:  0.13708333333333333
Avg avgDis:  1.5664166666666666

Old method:
Avg dis:  9.494
Avg avg:  0.14475000000000027
Avg avgDis:  1.77608333333333


Test number:  2

Summary 

phi:  0.9

num of voters:  30

num of winners:  2

num_of_reviews_per_candidate:  5

num of first votes:  2

num_of_second_votes:  3

num_of_first_votes_winners:  1

num_of_first_votes_losers:  1

prob_for_not_rank:  0.3




Best method:
Avg dis:  0.0
Avg avg:  3.0
Avg avgDis:  0.0

all Vote method:
Avg dis:  4.626
Avg avg:  0.588
Avg avgDis: 

In [ ]:
phi = [0.75, 0.8, 0.85, 0.9]
num_of_voters = [20, 30, 40, 50, 60, 70, 80]
winners_percentage = [5, 10, 15, 20, 25, 30, 35]
num_of_reviews_per_candidate=[5, 7, 9, 11]
split_step=[0.5, 0.75]
first_step_winners=[0.5, 0.75]
first_step_losers=[0.5, 0.75]
prob_for_not_rank = [0.1, 0.3, 0.5, 0.7]



num_of_iterations = 1000
num_of_tests = 50

color = ""

num_of_new_method_win = 0
num_of_old_method_win = 0
num_of_no_winner = 0

success_params = []
failed_params = []

for test_num in range(1, num_of_tests+1):
  
  best_avg_dis = 0
  best_avg_avg = 0
  best_avg_avg_Dis = 0

  all_vote_avg_dis = 0
  all_vote_avg_avg = 0
  all_vote_avg_avg_Dis = 0

  new_avg_dis = 0
  new_avg_avg = 0
  new_avg_avg_Dis = 0

  old_avg_dis = 0
  old_avg_avg = 0
  old_avg_avg_Dis = 0


  phi_val = random.choice(phi)
  num_of_voters_val = random.choice(num_of_voters)
  winners_percentage_val = random.choice(winners_percentage)
  num_of_reviews_per_candidate_val = random.choice(num_of_reviews_per_candidate)
  split_step_val = random.choice(split_step)
  first_step_winners_val = random.choice(first_step_winners)
  first_step_losers_val = random.choice(first_step_losers)
  prob_val = random.choice(prob_for_not_rank)


  for i in range(num_of_iterations):
    #print("\nIteartion number ", i, "\n")
    partition_class_test = partition(2,num_of_voters=num_of_voters_val,truthful_ranking=np.array(range(1,num_of_voters_val+1)),sigma=phi_val,winners_percentage=winners_percentage_val,num_of_reviews_per_candidate=num_of_reviews_per_candidate_val,split_step=split_step_val,first_step_winners=first_step_winners_val,first_step_losers=first_step_losers_val,prob_for_not_rank=prob_val,new_method=True)
    best_method_winners, all_votes, new_method_winners, old_method_winners = partition_class_test.train_vote()
    #print("new_method_winners: ", new_method_winners, " best_method_winners: ", best_method_winners, "all_votes: ", all_votes)

    best_avg, best_dis, best_avg_Dis = partition_class_test.accuracy(best_method_winners).values()
    best_avg_dis+= best_dis
    best_avg_avg+= best_avg
    best_avg_avg_Dis += best_avg_Dis

    all_vote_avg, all_vote_dis, all_vote_avg_Dis = partition_class_test.accuracy(all_votes).values()
    all_vote_avg_dis+= all_vote_dis
    all_vote_avg_avg+= all_vote_avg
    all_vote_avg_avg_Dis += all_vote_avg_Dis

    new_avg, new_dis, new_avg_Dis = partition_class_test.accuracy(new_method_winners).values()
    new_avg_dis+= new_dis
    new_avg_avg+= new_avg
    new_avg_avg_Dis += new_avg_Dis
    #print("dis ", new_dis, " avg ", new_avg, " avgDis ", new_avg_Dis)

    old_avg, old_dis, old_avg_Dis = partition_class_test.accuracy(old_method_winners).values()
    old_avg_dis+= old_dis
    old_avg_avg+= old_avg
    old_avg_avg_Dis += old_avg_Dis
  if (new_avg_dis < old_avg_dis):
    success_params.append({'sigma:': phi_val, 'num_of_voters: ' : num_of_voters_val, 'winners_percentage: ' : winners_percentage_val, 'num_of_reviews_per_candidate: ' : num_of_reviews_per_candidate_val, 'split_step: ' : split_step_val ,'first_step_winners: ' : first_step_winners_val, 'first_step_losers: ' : first_step_losers_val, 'prob_val: ' : prob_val })
    
    num_of_new_method_win += 1
    color = '\033[92m'
  elif (new_avg_dis == old_avg_dis):

    num_of_no_winner+=1
    color = '\033[93m'
  else:
    failed_params.append({'sigma:': phi_val, 'num_of_voters: ' : num_of_voters_val, 'winners_percentage: ' : winners_percentage_val, 'num_of_reviews_per_candidate: ' : num_of_reviews_per_candidate_val, 'split_step: ' : split_step_val ,'first_step_winners: ' : first_step_winners_val, 'first_step_losers: ' : first_step_losers_val})

    num_of_old_method_win += 1
    color = '\033[91m'
  print("\n")
  print(color+"Test number: ", test_num)
  #print("\nSummary for phi: ", phi_val, " and Num of iterations: ", num_of_iterations, " num of winners: ", partition_class_test.num_of_winners, " num of first votes: ", partition_class_test.num_of_first_votes)
  partition_class_test.print_test_summary()
  print("Best method:")
  print("Avg dis: ", best_avg_dis/num_of_iterations)
  print("Avg avg: ", best_avg_avg/num_of_iterations)
  print("Avg avgDis: ", best_avg_avg_Dis/num_of_iterations)

  print("\nall Vote method:")
  print("Avg dis: ", all_vote_avg_dis/num_of_iterations)
  print("Avg avg: ", all_vote_avg_avg/num_of_iterations)
  print("Avg avgDis: ", all_vote_avg_avg_Dis/num_of_iterations)

  print("\nNew method:")
  print("Avg dis: ", new_avg_dis/num_of_iterations)
  print("Avg avg: ", new_avg_avg/num_of_iterations)
  print("Avg avgDis: ", new_avg_avg_Dis/num_of_iterations)

  print("\nOld method:")
  print("Avg dis: ", old_avg_dis/num_of_iterations)
  print("Avg avg: ", old_avg_avg/num_of_iterations)
  print("Avg avgDis: ", old_avg_avg_Dis/num_of_iterations)

  

print('\033[0m' + "Number of time new method was better: ", num_of_new_method_win, " number of time old method was better: ", num_of_old_method_win, " number of time of same score: ", num_of_no_winner)
print(success_params)
print("/n/n/n/n")
print(failed_params)



Test number:  1

Summary 

phi:  0.9

num of voters:  30

num of winners:  4

num_of_reviews_per_candidate:  9

num of first votes:  7

num_of_second_votes:  2

num_of_first_votes_winners:  3

num_of_first_votes_losers:  2

prob_for_not_rank:  0.7




Best method:
Avg dis:  0.0
Avg avg:  1.0
Avg avgDis:  0.0

all Vote method:
Avg dis:  8.003
Avg avg:  0.4135
Avg avgDis:  4.064

New method:
Avg dis:  9.415
Avg avg:  0.446
Avg avgDis:  5.028

Old method:
Avg dis:  9.757
Avg avg:  0.4555
Avg avgDis:  5.32275


Test number:  2

Summary 

phi:  0.85

num of voters:  20

num of winners:  6

num_of_reviews_per_candidate:  7

num of first votes:  6

num_of_second_votes:  1

num_of_first_votes_winners:  5

num_of_first_votes_losers:  5

prob_for_not_rank:  0.7




Best method:
Avg dis:  0.274
Avg avg:  0.3603333333333311
Avg avgDis:  0.1425

all Vote method:
Avg dis:  7.246
Avg avg:  0.2588333333333335
Avg avgDis:  2.4031666666666642

New method:
Avg dis:  7.755
Avg avg:  0.26999999999999996


Old CODE

In [ ]:
# ###########Test partition class###########

# ##Test voter ranking
# print("Test voter ranking:")
# print("Expected [1..200]")
# partition_class_test = partition(2,200,range(1,201),0,3)
# print(partition_class_test.voter_sampled_ranks)  ##expected [1..200]
# print(partition_class_test.voter_sampled_ranks[50]) ##expected [1..200]
# print("\nExpected [1..100]")
# partition_class_test = partition(2,100,range(1,101),0,3)
# print(partition_class_test.voter_sampled_ranks[0])  ##expected [1..100]
# print(partition_class_test.voter_sampled_ranks[50]) ##expected [1..100]
# print("\nExpected [random values]")
# partition_class_test = partition(2,200,range(1,201),1,3)
# print(partition_class_test.voter_sampled_ranks[0])  ##expected [random values]
# print(partition_class_test.voter_sampled_ranks[50]) ##expected [random values]
# print("\nExpected [close to ground truthful ranking]")
# partition_class_test = partition(2,200,range(1,201),0.5,3)
# print(partition_class_test.voter_sampled_ranks[0])  ##expected [random values]
# print(partition_class_test.voter_sampled_ranks[50])


In [ ]:
# ##Test train 
# print("\nTest train:")
# partition_class_test = partition(2,200,np.array(range(1,201)),0,3)
# partition_class_test.train()
# print("3 first voters in partition 1 : ", partition_class_test.voters_sampled_rank_by_partition[0][:3])
# print("3 first voters in partition 2 : ", partition_class_test.voters_sampled_rank_by_partition[1][:3])
# print("\n")

# partition_class_test = partition(2,200,np.array(range(1,201)),1,3)
# partition_class_test.train()
# print("\n3 first voters in partition 1 : ", partition_class_test.voters_sampled_rank_by_partition[0][:3])
# print("3 first voters in partition 2 : ", partition_class_test.voters_sampled_rank_by_partition[1][:3])


In [ ]:
# partition_class_test = partition(2,200,np.array(range(1,201)),0,3)
# test_array = np.array(range(1,201))
# ##Test get_winners_percentage_from_voter_sampled_rank
# print("Test get_winners_percentage_from_voter_sampled_rank")
# print("Expect value: " , test_array[:6])
# print("Real Value: " , get_winners_percentage_from_voter_sampled_rank(test_array, 3))
# print("Shuffle ranking...")
# np.random.shuffle(test_array)
# print("Expect value: " , test_array[:6])
# print("Real Value: " ,get_winners_percentage_from_voter_sampled_rank(test_array, 3))

# ##Test get_partition_winners
# print("\nget_partition_winners")
# test_array = [[1,2,3], [1,2,4], [1,2,3]]
# print("Expect value: [1,2,3]")
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))


# test_array = [[1,2,4], [1,2,5], [1,2,3]]
# print("\nExpect value: [1,2,3/4/5] will choose randomly")
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))

# test_array = [[1,2], [3,4], [5,6]]
# print("\nExpect value:  will choose 2 numbers randomly")
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))
# print("Real Value: " , partition_class_test.get_partition_winners(test_array))

# ##Test Vote
# print("\nTest Vote")
# print("truthful_ranking: ", partition_class_test.truthful_ranking)
# partition_class_test.train()
# print("partition 1 winners: ", partition_class_test.Voter_partition_i[0][:3])
# print("partition 2 winners: ", partition_class_test.Voter_partition_i[1][:3])
# print("Partition vote: ",partition_class_test.vote())

# partition_class_test.train()
# print("\npartition 1 winners: ", partition_class_test.Voter_partition_i[0][:3])
# print("partition 2 winners: ", partition_class_test.Voter_partition_i[1][:3])
# print("Partition vote: ",partition_class_test.vote())

# partition_class_test = partition(2,200)
# print("\ntruthful_ranking: ", partition_class_test.truthful_ranking)
# partition_class_test.train()
# print("partition 1 winners: ", partition_class_test.Voter_partition_i[0][:3])
# print("partition 2 winners: ", partition_class_test.Voter_partition_i[1][:3])
# print("Partition vote: ",partition_class_test.vote())

# partition_class_test.train()
# print("\npartition 1 winners: ", partition_class_test.Voter_partition_i[0][:3])
# print("partition 2 winners: ", partition_class_test.Voter_partition_i[1][:3])
# print("Partition vote: ",partition_class_test.vote())

In [ ]:
# debug=0
# num_of_voters = 200
# winners_percentage = 3  ##3%
# truthful_ranking = np.array(range(1,num_of_voters+1)) ##truthful_ranking [1>2>3>4...>199>200]
# sigma = 0 
# partition_class = partition(2,num_of_voters,truthful_ranking,sigma,winners_percentage)   ##currently support only for partition of 2

# partition_class.train()
# print("Vote :", partition_class.vote())
# print("Vote 50 score:", partition_class.train_vote_x_times_score(200))
# print("Summary :")
# print("Unique: ", partition_class.iteration_i_unique[0])
# print("Counts: ", partition_class.iteration_i_counts[0])
# print("Avg winners: ", partition_class.itreration_train_vote_x_times[0])
# print("Vote 50 AVG miss:", partition_class.train_vote_x_times_score(200, metrics=["avg"]))
# print("Summary :")
# print("Unique: ", partition_class.iteration_i_unique[0])
# print("Counts: ", partition_class.iteration_i_counts[0])
# print("Avg winners: ", partition_class.itreration_train_vote_x_times[0])




In [ ]:
# #######Question number 2#######
# print("Question number 2:")
# num_of_voters = 200
# winners_percentage = 3  ##3%
# truthful_ranking = np.array(range(1,num_of_voters+1)) ##truthful_ranking [1>2>3>4...>199>200]
# sigma = 0 

# voters_ranks_best_method = create_n_rannking_from_truthful_ranking_and_sigma(truthful_ranking, sigma=sigma)

# best_method = get_winners_percentage_from_voter_sampled_rank(voters_ranks_best_method, winners_percentage)

# print("Best method for Num of voters =", num_of_voters, "and", winners_percentage ,"% winners is ", best_method)

In [ ]:
# #######Question number 3#######
# print("Question number 3:")
# num_of_voters = 200
# winners_percentage = 3  ##3%
# truthful_ranking = np.array(range(1,num_of_voters+1)) ##truthful_ranking [1>2>3>4...>199>200]
# sigma = 0 
# partition_class = partition(2,num_of_voters,truthful_ranking,sigma,winners_percentage)   ##currently support only for partition of 2

# print("Partition vote with sigma=", sigma, "is", partition_class.train_vote())

# sigma = 0.8
# partition_class = partition(2,num_of_voters,truthful_ranking,sigma,winners_percentage)   ##currently support only for partition of 2
# print("\nPartition vote with sigma=", sigma, "is", partition_class.train_vote())

# partition_vote = partition_class.train_vote()
# print("\nPartition vote with sigma=", sigma, "is", partition_vote, "and accuracy=", partition_class.accuray(partition_vote))


In [ ]:
# #######Question number 4#######
# print("Question number 4:")
# num_of_voters = 200
# winners_percentage = 3  ##3%
# truthful_ranking = np.array(range(1,num_of_voters+1)) ##truthful_ranking [1>2>3>4...>199>200]
# sigma = 0.8
# partition_class = partition(2,num_of_voters,truthful_ranking,sigma,winners_percentage)   ##currently support only for partition of 2

# print("Vote 50 score:", partition_class.train_vote_x_times_score(200))
# print("Summary :")
# print("Unique: ", partition_class.iteration_i_unique[0])
# print("Counts: ", partition_class.iteration_i_counts[0])
# print("Avg winners: ", partition_class.itreration_train_vote_x_times[0])




In [ ]:
# #######Question number 5#######
# print("Question number 5:")
# num_of_voters = 200
# winners_percentage = 3  ##3%
# truthful_ranking = np.array(range(1,num_of_voters+1)) ##truthful_ranking [1>2>3>4...>199>200]
# sigma = 0.8
# partition_class = partition(2,num_of_voters,truthful_ranking,sigma,winners_percentage)   ##currently support only for partition of 2

# print(partition_class.train_vote_x_times_score_resmapled(num_of_iterations=1,resmapled_every=200,best_method=[],metrics=["avg","dis", "avgDis"]))

In [ ]:
##Summary:
# 1. לתת משקל שונה לכל הרצה